In [ ]:
# https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.width', 5000)

In [3]:
col_dtypes = {
    #'client_id': np.int64,
    #'loan_type': str
}
#dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

train = pd.read_csv("../data/house-prices/train.csv",delimiter=",")
sub = pd.read_csv("../data/house-prices/test.csv",delimiter=",")#,parse_dates=['loan_start','loan_end'], date_parser=dateparse, dtype=loans_dtypes)

In [4]:
sub.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
Id               1459 non-null int64
MSSubClass       1459 non-null int64
MSZoning         1455 non-null object
LotFrontage      1232 non-null float64
LotArea          1459 non-null int64
Street           1459 non-null object
Alley            107 non-null object
LotShape         1459 non-null object
LandContour      1459 non-null object
Utilities        1457 non-null object
LotConfig        1459 non-null object
LandSlope        1459 non-null object
Neighborhood     1459 non-null object
Condition1       1459 non-null object
Condition2       1459 non-null object
BldgType         1459 non-null object
HouseStyle       1459 non-null object
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
YearBuilt        1459 non-null int64
YearRemodAdd     1459 non-null int64
RoofStyle        1459 non-null object
RoofMatl         1459 non-null object
Exterior1st      1458 non-

In [5]:
# use some automl
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-8u232-b09-0ubuntu1~18.04.1-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /home/tk/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpgotgf0fy
  JVM stdout: /tmp/tmpgotgf0fy/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpgotgf0fy/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Etc/GMT
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_unknownUser_7sv6k9
H2O cluster total nodes:,1
H2O cluster free memory:,3.417 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [6]:
# Load a pandas data frame to H2O
hf = h2o.H2OFrame(train)
hf_sub = h2o.H2OFrame(sub)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
x = hf.names
x.remove("SalePrice")
Y = "SalePrice"

In [8]:
train, test = hf.split_frame([0.7], seed=42)

In [11]:
from h2o.estimators.xgboost import H2OXGBoostEstimator
xgb = H2OXGBoostEstimator(nfolds=10, seed=1)
xgb.train(x=x, y=Y, training_frame=train,
         validation_frame=test)

print(xgb)

xgboost Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1580089390761_1


Model Summary: 


,,number_of_trees
0,,50.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 14227052.298257211
RMSE: 3771.876495626177
MAE: 2736.378879651021
RMSLE: 0.02518994254098492
Mean Residual Deviance: 14227052.298257211

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 1030312372.9361557
RMSE: 32098.479293202592
MAE: 18771.471254701966
RMSLE: 0.15000402224513062
Mean Residual Deviance: 1030312372.9361557

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 955123746.8223003
RMSE: 30905.076392435923
MAE: 18173.18750759971
RMSLE: 0.14554037255646643
Mean Residual Deviance: 955123746.8223003

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
0,mae,18191.447,2011.4663,18264.902,18711.209,16423.943,16052.271,16783.9,22195.51,20729.475,18961.238,16765.488,17026.527
1,mean_residual_deviance,9.5814202E8,5.5282342E8,6.8985197E8,6.6610477E8,7.3492307E8,6.7142272E8,7.9649472E8,2.15875456E9,9.5534362E8,1.77933478E9,4.95678016E8,6.3351206E8
2,mse,9.5814202E8,5.5282342E8,6.8985197E8,6.6610477E8,7.3492307E8,6.7142272E8,7.9649472E8,2.15875456E9,9.5534362E8,1.77933478E9,4.95678016E8,6.3351206E8
3,r2,0.84413165,0.08404689,0.87013525,0.84683806,0.8295289,0.88145614,0.90560186,0.63004947,0.8836042,0.7882095,0.9089278,0.8969651
4,residual_deviance,9.5814202E8,5.5282342E8,6.8985197E8,6.6610477E8,7.3492307E8,6.7142272E8,7.9649472E8,2.15875456E9,9.5534362E8,1.77933478E9,4.95678016E8,6.3351206E8
5,rmse,30030.426,7910.301,26265.033,25809.006,27109.465,25911.826,28222.238,46462.4,30908.633,42182.16,22263.828,25169.666
6,rmsle,0.14416508,0.021460833,0.15368767,0.14617883,0.14133656,0.12610854,0.11954388,0.17885941,0.17726208,0.13013846,0.14873068,0.11980466



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2020-01-27 01:59:11,3.379 sec,0.0,197684.542928,181167.142996,3.907918e+10,197341.539793,180334.243056,3.894368e+10
1,,2020-01-27 01:59:11,3.398 sec,1.0,141710.753370,127772.409698,2.008194e+10,142587.509786,126857.109949,2.033120e+10
2,,2020-01-27 01:59:11,3.408 sec,2.0,102243.171067,90398.704143,1.045367e+10,105656.007693,89935.362576,1.116319e+10
3,,2020-01-27 01:59:11,3.419 sec,3.0,74485.045961,64027.462499,5.548022e+09,79335.704034,63788.278185,6.294154e+09
4,,2020-01-27 01:59:11,3.431 sec,4.0,54850.705854,45559.081545,3.008600e+09,61081.138779,46316.405111,3.730906e+09
5,,2020-01-27 01:59:11,3.443 sec,5.0,41038.548359,32601.949971,1.684162e+09,49933.237705,35365.011891,2.493328e+09
6,,2020-01-27 01:59:11,3.454 sec,6.0,31139.993676,23681.207065,9.696992e+08,42513.106588,28272.252089,1.807364e+09
7,,2020-01-27 01:59:11,3.466 sec,7.0,24283.244244,17683.560402,5.896760e+08,38209.109015,24128.990361,1.459936e+09
8,,2020-01-27 01:59:11,3.477 sec,8.0,19329.140509,13787.736184,3.736157e+08,35356.390732,21711.893645,1.250074e+09
9,,2020-01-27 01:59:11,3.489 sec,9.0,15879.497227,11267.017206,2.521584e+08,34008.117223,20535.764621,1.156552e+09



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,OverallQual,6.160809e+12,1.000000,0.559075
1,GrLivArea,1.558908e+12,0.253036,0.141466
2,GarageArea,8.529537e+11,0.138448,0.077403
3,TotalBsmtSF,3.520164e+11,0.057138,0.031944
4,2ndFlrSF,2.610313e+11,0.042370,0.023688
5,1stFlrSF,2.234672e+11,0.036272,0.020279
6,FullBath,2.232107e+11,0.036231,0.020256
7,BsmtFinSF1,1.608038e+11,0.026101,0.014592
8,LotArea,1.530725e+11,0.024846,0.013891
9,KitchenQual.Ex,9.837817e+10,0.015968,0.008928



See the whole table with table.as_data_frame()



In [45]:
# Grid Search
from h2o.grid.grid_search import H2OGridSearch

xgb_parameters = {'max_depth': [3, 4, 5, 6],
                  'sample_rate': [0.7, 0.8],
                  'col_sample_rate': [0.7, 0.8],
                  'ntrees': [200, 300, 400]}

xgb_grid_search = H2OGridSearch(model=H2OXGBoostEstimator,
                                grid_id='example_grid',
                                hyper_params=xgb_parameters)

xgb_grid_search.train(x=x,
                      y=Y,
                      training_frame=train,
                      validation_frame=test,
                      nfolds = 10,
                      learn_rate=0.01,
                      seed=42)

xgboost Grid Build progress: |████████████████████████████████████████████| 100%


In [47]:
grid_results = xgb_grid_search.get_grid(sort_by='rmse',
                                        decreasing=False)
print(grid_results)

       col_sample_rate max_depth ntrees sample_rate               model_ids                rmse
0                  0.8         3    300         0.9  example_grid_model_243     26006.880308855
1                  0.8         3    200         0.9  example_grid_model_234  26104.934073280896
2                  0.8         4    300         0.9  example_grid_model_246  26187.529742584964
3                  0.8         4    200         0.9  example_grid_model_237   26230.37344947747
4                  0.8         5    300         0.9  example_grid_model_249  26249.632679731752
..  ..             ...       ...    ...         ...                     ...                 ...
329                0.8         3    200         0.8  example_grid_model_312  42941.692605012395
330                0.7         3    200         0.8  example_grid_model_311   43007.97992669297
331               0.06         4     50        0.02   example_grid_model_20   43068.81825870879
332                0.8         3    200 

In [41]:
xgb = grid_results.models[0]

preds = xgb.predict(hf_sub)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [48]:
from h2o.automl import H2OAutoML
autoML = H2OAutoML(max_runtime_secs=3600, 
            nfolds = 10,            
            stopping_metric = 'RMSE',
            sort_metric = 'RMSE',
            exclude_algos=['DRF','GLM'],
        )

autoML.train(x=x,
             y=Y,
             training_frame=train,
             validation_frame=test,
        )

print(autoML.leaderboard)

AutoML progress: |
03:19:02.997: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%| 100%


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200127_031902,26292.5,6.91297e+08,6.91297e+08,15154.9,0.12446
GBM_grid__1_AutoML_20200127_031902_model_24,26687.5,7.12223e+08,7.12223e+08,16066.1,0.131692
GBM_4_AutoML_20200127_031902,26950.3,7.2632e+08,7.2632e+08,15839.1,0.128558
XGBoost_grid__1_AutoML_20200127_031902_model_6,27309.8,7.45823e+08,7.45823e+08,16016.5,0.130454
GBM_grid__1_AutoML_20200127_031902_model_1,27321.6,7.46469e+08,7.46469e+08,15769.2,0.129328
XGBoost_3_AutoML_20200127_031902,27414.6,7.51561e+08,7.51561e+08,15593.3,0.131179
XGBoost_2_AutoML_20200127_031902,27492.6,7.55841e+08,7.55841e+08,15733.2,0.129585
XGBoost_grid__1_AutoML_20200127_031902_model_3,27515.4,7.57095e+08,7.57095e+08,15761,0.132977
GBM_2_AutoML_20200127_031902,27581.5,7.6074e+08,7.6074e+08,15845.7,0.128463
XGBoost_grid__1_AutoML_20200127_031902_model_8,27614.1,7.62537e+08,7.62537e+08,15993.9,0.13287


In [35]:
import h2o

preds = autoML.leader.predict(hf_sub)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [42]:
submission = hf_sub['Id']

In [43]:
submission['SalePrice'] = preds

In [44]:
h2o.h2o.download_csv(submission, "../data/house-prices/submission.csv")

'/workspaces/aws_ml/data/house-prices/submission.csv'